In [ ]:
#importing libraries

import re
import pickle
import numpy as np
import pandas as pd

# plotting

import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt|


# nltk
import nltk as nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from gensim.models import Word2Vec
from sklearn.decomposition import PCA


#spacy

import spacy
!pip install spacy
spacy.cli.download("en_core_web_sm")
# Load the English language model
nlp = spacy.load('en_core_web_sm')

#scaling
from sklearn.preprocessing import MinMaxScaler

#model building

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


In [ ]:
#importing the file
df = pd.read_csv("E:\\NLP PROJECTS\\extra\\twitter_training.csv",encoding='ISO-8859-1')
df

In [ ]:
#giving columns names
#bot means base of  topic
df.columns = ['Tweet_id','BOT','Sentiments','Tweet_content']
df.head()

In [ ]:
#checking null values
df.isnull().sum()

In [ ]:
#droping null values rows and renaming the new data columns
df1 =df.dropna(axis=0)
df1.columns = ['Tweet_id','BOT','Sentiments','Tweet_content']

In [ ]:
#rechecking null values
df1.isnull().sum()

In [ ]:

#1 lowercasing all elements
df1['clean_text'] = df1['Tweet_content'].apply(lambda x: x.lower())

#2 removing urls
def remove_urls(text):
    return re.sub('https.............\S+', '', text)
    return re.sub('bit.......\S+' '', text)

df1['clean_text'] = df1['Tweet_content'].apply(remove_urls)
 
    
    #3 removing special characters and puntuations
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

df1['clean_text'] = df1['Tweet_content'].apply(remove_special_characters)

In [ ]:
df1

In [ ]:

# Define contractions
contractions = {
    "im" : "i am",
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "we'll": "we will"
    
}
# Set of English stop words
stop_words = set(stopwords.words('english'))

def process_text(text):
    # Join the list of tokens into a single string
    if isinstance(text, list):
        text = ' '.join(text)
    
    # Remove contractions
    for contraction, expansion in contractions.items():
        text = re.sub(r'\b' + contraction + r'\b', expansion, text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemmatize the remaining tokens using spaCy
    doc = nlp(" ".join(filtered_tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]
    
    # Join the lemmatized tokens into a string
    return ' '.join(lemmatized_tokens)

# Apply the processing function to the 'clean_text' column of your DataFrame
df1['clean_text'] = df1['clean_text'].apply(process_text)
df1

In [ ]:
df1['clean_text']


In [ ]:

# Extracting tokenized clean_text column
tokenized_clean_text_data = df1['clean_text'].tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_clean_text_data, vector_size=100, window=5, min_count=1, workers=4)

# Create word vectors
word_vectors = word2vec_model.wv


# Convert tokenized data to word vectors
word_vector_data = []
for text in tokenized_clean_text_data:
    vectors = [word_vectors[word] for word in text if word in word_vectors]
    if vectors:
        document_vector = sum(vectors) / len(vectors)
        word_vector_data.append(document_vector)
    else:
        # Handle empty text case
        word_vector_data.append([0] * word2vec_model.vector_size)

# Convert document vectors to DataFrame (optional)
document_vectors_df = pd.DataFrame(word_vector_data)
print(document_vectors_df)


In [ ]:

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit scaler to data and transform
document_vectors_normalized = scaler.fit_transform(document_vectors_df)

# Now 'document_vectors_normalized' contains normalized vectors with values between 0 and 1


In [ ]:


# Extracting clean_text column
clean_text_data = df1['clean_text'].tolist()
labels = df1['Sentiments']  

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(clean_text_data, labels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train SVM model
svm_model = SVC(kernel='linear')  # You can change the kernel as needed
svm_model.fit(X_train_vectorized, y_train)

# Predictions using SVM model
svm_predictions = svm_model.predict(X_test_vectorized)

# Calculate accuracy for SVM model
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

# Calculate F1 score for SVM model
svm_f1_score = f1_score(y_test, svm_predictions, average='weighted')
print("SVM F1 Score:", svm_f1_score)

# Confusion matrix for SVM model
svm_confusion_matrix = confusion_matrix(y_test, svm_predictions)
print("SVM Confusion Matrix:")
print(svm_confusion_matrix)


In [ ]:
# Extracting clean_text column and labels
clean_text_data = df1['clean_text'].tolist()
labels = df1['Sentiments']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(clean_text_data, labels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Define the SVM model
svm_model = SVC()

# Define parameter grid for GridSearchCV
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']  # Only relevant for 'rbf' and 'poly' kernels
}

# Initialize GridSearchCV
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_vectorized, y_train)

# Get the best parameters and best estimator from grid search
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters:", best_params)

# Predict using the best estimator
svm_predictions = best_estimator.predict(X_test_vectorized)

# Calculate accuracy for the best estimator
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

# Calculate F1 score for the best estimator
svm_f1_score = f1_score(y_test, svm_predictions, average='weighted')
print("SVM F1 Score:", svm_f1_score)

# Confusion matrix for the best estimator
svm_confusion_matrix = confusion_matrix(y_test, svm_predictions)
print("SVM Confusion Matrix:")
print(svm_confusion_matrix)